# Orchestration Workflow

## Install Library

In [ ]:
%pip install azure-ai-language-conversations

## Load Azure Configurations

In [1]:
import os

# Load Azure configurations from environment variables
# Ensure that AZURE_AI_LANGUAGE_KEY and AZURE_AI_LANGUAGE_ENDPOINT are set in your environment
language_key = os.environ.get('AZURE_AI_LANGUAGE_KEY')
language_endpoint = os.environ.get('AZURE_AI_LANGUAGE_ENDPOINT')

# Represents the Project Name
project_name = "Python-Orchestration-Challenge"
# Represents the output model label.
modelLabel = "Python-Orchestration-Model-Challenge"  

## Create ConversationAuthoringClient

In [2]:
from azure.ai.language.conversations.authoring import ConversationAuthoringClient
from azure.core.credentials import AzureKeyCredential

# Authenticate the client using Azure Key and Endpoint
def authenticate_client():
    """
    Authenticates the Azure ConversationAuthoringClient using the provided key and endpoint.

    Returns:
        ConversationAuthoringClient: An authenticated client for Azure Conversation Authoring.
    """
    credential = AzureKeyCredential(language_key)
    client = ConversationAuthoringClient(
        endpoint=language_endpoint,
        credential=credential
    )
    return client

# Initialize the client
conversation_authoring_client = authenticate_client()

## Create a Project

In [3]:
project = {
        "language": "en-us",  # The project language. This is BCP-47 representation of a language. For example, use "en" for English, "en-gb" for English (UK), "es" for Spanish etc. Required.
        "projectKind": "Orchestration",  # Represents the project kind. Required. Known values are: "Conversation" and "Orchestration".
        "projectName": project_name,  # The new project name. Required.
        "description": "Orchestration Workflow Challenge",  # Optional. The project description.
        "multilingual": False,  # Optional. Whether the project would be used for multiple languages or not.
        "settings": {
           "confidenceThreshold": 0.0,  # The threshold of the intent with the highest confidence, at which the prediction will automatically be changed to None. The value of the threshold should be between 0 and 1 inclusive. Required.
       }
   }

conversation_authoring_client.create_project(project_name, project)

{'createdDateTime': '2025-04-15T12:38:48Z',
 'lastModifiedDateTime': '2025-04-15T12:38:48Z',
 'projectKind': 'Orchestration',
 'settings': {'confidenceThreshold': 0.0,
  'normalizeCasing': False,
  'augmentDiacritics': False},
 'projectName': 'Python-Orchestration-Challenge',
 'multilingual': False,
 'description': 'Orchestration Workflow Challenge',
 'language': 'en-us'}

## Define Project Assets and Intents

In [4]:
exported_project_assets = {
    "projectKind": "Orchestration",
    "intents": [
        {
            "category": "None"
        },
        {
            "category": "Intent_CLU",
            "orchestration": {
                "targetProjectKind": "Conversation",
                "conversationOrchestration": {
                    "projectName": "CLU_Challenge",
                    "deploymentName": "Travel_CLU"
                }
            }
        },
        {
            "category": "Intent_QA",
            "orchestration": {
                "targetProjectKind": "QuestionAnswering",
                "questionAnsweringOrchestration": {
                    "projectName": "TravelFAQ"
                }
            }
        }
    ],
    "entities": [],
    "utterances": []
}

## Import Project

In [5]:
poller = conversation_authoring_client.begin_import_project(
    project_name=project_name,
    project={
        "assets": exported_project_assets,
        "metadata": {
            "projectKind": "Orchestration",
            "settings": {
                "confidenceThreshold": 0.0,
            },
            "projectName": project_name,
            "multilingual": False,
            "language": "en-us",
        },
        "projectFileVersion": "2022-05-01",
        "stringIndexType": "Utf16CodeUnit"  # Add this line to specify the StringIndexType
    },
)
response = poller.result()
print(response)

{'jobId': '0a3c25f6-5875-4c1a-9e34-97cfd1d90266_638802720000000000', 'createdDateTime': '2025-04-15T12:40:43Z', 'lastUpdatedDateTime': '2025-04-15T12:40:43Z', 'expirationDateTime': '2025-04-22T12:40:43Z', 'status': 'succeeded'}


## Train Model

In [6]:
# JSON input template you can fill out and use as your body input.
configuration = {
    "modelLabel": modelLabel,  # Represents the output model label. Required.
    "trainingMode": "standard",  # Represents the mode of the training operation. Required. Known values are: "advanced" and "standard".
    "evaluationOptions": {
        "kind": "percentage",  # Optional. Represents the evaluation kind. By default, the evaluation kind is set to percentage. Known values are: "percentage" and "manual".
        "testingSplitPercentage": 20,  # Optional. Represents the testing dataset split percentage. Only needed in case the evaluation kind is percentage.
        "trainingSplitPercentage": 80  # Optional. Represents the training dataset split percentage. Only needed in case the evaluation kind is percentage.
    },
    #"trainingConfigVersion": "str"  # Optional. Represents training config version. By default, "latest" value is used which uses the latest released training config version.
   }

poller = conversation_authoring_client.begin_train(project_name=project_name,configuration=configuration)
response = poller.result()
print(response)

{'result': {'modelLabel': 'Python-Orchestration-Model-Challenge', 'trainingConfigVersion': '2022-05-01', 'trainingMode': 'standard', 'trainingStatus': {'percentComplete': 100, 'startDateTime': '2025-04-15T12:40:54.3641233Z', 'endDateTime': '2025-04-15T12:40:56.7044941Z', 'status': 'succeeded'}, 'evaluationStatus': {'percentComplete': 100, 'startDateTime': '2025-04-15T12:40:57.451491Z', 'endDateTime': '2025-04-15T12:41:13.1068155Z', 'status': 'succeeded'}}, 'jobId': '23c46502-f7e3-4994-804f-ceef70b31f6f_638802720000000000', 'createdDateTime': '2025-04-15T12:40:52Z', 'lastUpdatedDateTime': '2025-04-15T12:41:57Z', 'expirationDateTime': '2025-04-22T12:40:52Z', 'status': 'succeeded', 'warnings': []}


## Display Model Evaluation Summary

In [7]:
evaluation_summary = conversation_authoring_client.get_model_evaluation_summary(project_name=project_name, trained_model_label=modelLabel)
print(evaluation_summary)

{'intentsEvaluation': {'confusionMatrix': {'None': {}, 'Intent_CLU': {'Intent_CLU': {'normalizedValue': 90.0, 'rawValue': 9.0}, 'Intent_QA': {'normalizedValue': 10.0, 'rawValue': 1.0}}, 'Intent_QA': {'Intent_QA': {'normalizedValue': 85.71429, 'rawValue': 6.0}, 'Intent_CLU': {'normalizedValue': 14.285714, 'rawValue': 1.0}}}, 'intents': {'Intent_CLU': {'f1': 0.8999999761581421, 'precision': 0.8999999761581421, 'recall': 0.8999999761581421, 'truePositiveCount': 9, 'trueNegativeCount': 6, 'falsePositiveCount': 1, 'falseNegativeCount': 1}, 'Intent_QA': {'f1': 0.8571428656578064, 'precision': 0.8571428656578064, 'recall': 0.8571428656578064, 'truePositiveCount': 6, 'trueNegativeCount': 9, 'falsePositiveCount': 1, 'falseNegativeCount': 1}, 'None': {'f1': 0.0, 'precision': 0.0, 'recall': 0.0, 'truePositiveCount': 0, 'trueNegativeCount': 0, 'falsePositiveCount': 0, 'falseNegativeCount': 0}}, 'microF1': 0.88235295, 'microPrecision': 0.88235295, 'microRecall': 0.88235295, 'macroF1': 0.8785714, 'm

## Deploy the Trained Model

In [8]:
deployment = {
       "trainedModelLabel": modelLabel  # Represents the trained model label. Required.
   }

trained_model = conversation_authoring_client.begin_deploy_project(project_name=project_name, deployment_name=modelLabel, deployment=deployment)
response = trained_model.result()
print(response)

{'deploymentName': 'Python-Orchestration-Model-Challenge', 'modelId': 'Python-Orchestration-Model-Challenge-20250415T124157-2055d755b4224cdb9165297ca62a0c89', 'lastTrainedDateTime': '2025-04-15T12:41:57.7969696Z', 'lastDeployedDateTime': '2025-04-15T12:43:23Z', 'deploymentExpirationDate': '2026-10-27', 'modelTrainingConfigVersion': '2022-05-01'}


## Making the Request


## Create a ConversationAnalysisClient

In [9]:
from azure.ai.language.conversations import ConversationAnalysisClient
from azure.core.credentials import AzureKeyCredential

credential = AzureKeyCredential(language_key)
conversation_analysis_client = ConversationAnalysisClient(
        endpoint=language_endpoint,
        credential=credential
)

## Helper function to Send requests

In [18]:
def display_result(result):
    """
    Processes and displays the result of an analyzed conversation.

    This function takes the result of a conversation analysis and extracts
    relevant information such as the top intent, project kind, and details
    about specific intents (e.g., Intent_CLU or Intent_QA). It prints the
    extracted information in a readable format.

    Args:
        result (dict): The result of the conversation analysis, typically
                       returned by the Azure ConversationAnalysisClient.

    Returns:
        None: The function prints the result details to the console.
    """
    # Check if the result is of type 'ConversationResult'
    if result.get('kind') == 'ConversationResult':
        prediction = result['result']['prediction']
        query = result['result']['query']
        top_intent = prediction['topIntent']
        project_kind = prediction['projectKind']

        # Print basic information about the query and prediction
        print(f"Query: {query}")
        print(f"Top Intent: {top_intent}")
        print(f"Project Kind: {project_kind}")

        # Handle details for the 'Intent_CLU' intent
        if top_intent == 'Intent_CLU':
            intent_details = prediction['intents']['Intent_CLU']
            print(f"\nIntent_CLU Details:")
            print(f"  Confidence Score: {intent_details['confidenceScore']}")
            if 'result' in intent_details:
                sub_prediction = intent_details['result']['prediction']
                print(f"  Sub-Intent: {sub_prediction['topIntent']}")
                print("  Sub-Intent Confidence Scores:")
                for sub_intent in sub_prediction['intents']:
                    print(f"    - {sub_intent['category']}: {sub_intent['confidenceScore']}")
                print("\nEntities:")
                for entity in sub_prediction['entities']:
                    print(f"  - {entity['category']}: {entity['text']}")
                    print(f"    Confidence Score: {entity['confidenceScore']}")
                    if 'resolutions' in entity:
                        print(f"    Resolutions: {entity['resolutions']}")
                    if 'extraInformation' in entity:
                        print(f"    Extra Information: {entity['extraInformation']}")

        # Handle details for the 'Intent_QA' intent
        elif top_intent == 'Intent_QA':
            intent_details = prediction['intents']['Intent_QA']
            print(f"\nIntent_QA Details:")
            print(f"  Confidence Score: {intent_details['confidenceScore']}")
            if 'result' in intent_details:
                answers = intent_details['result']['answers']
                print("\nAnswers:")
                for answer in answers:
                    print(f"  - Answer: {answer['answer']}")
                    print(f"    Confidence Score: {answer['confidenceScore']}")
                    print(f"    Source: {answer['source']}")
                    print(f"    Questions: {answer['questions']}")
                    if 'metadata' in answer:
                        print(f"    Metadata: {answer['metadata']}")
                    if 'dialog' in answer:
                        print(f"    Dialog Prompts:")
                        for prompt in answer['dialog']['prompts']:
                            print(f"      - {prompt['displayText']} (ID: {prompt['qnaId']})")

        # Handle cases where no relevant intent is found
        else:
            print("\nNo relevant intent found.")

    # Handle invalid result formats
    else:
        print("Invalid result format.")

def analyze_utterance(utterance):
    """
    Analyzes the given utterance using the deployed Azure AI Language model.

    This function sends the provided utterance to the Azure ConversationAnalysisClient
    for analysis. It uses the deployed model to predict the intent and entities
    associated with the utterance.

    Args:
        utterance (str): The text input to analyze.

    Returns:
        None: Prints the result of the analysis to the console.
    """
    # Send the utterance to the Azure ConversationAnalysisClient for analysis
    result = conversation_analysis_client.analyze_conversation(
        task={
            "kind": "Conversation",  # Specify the task type as "Conversation"
            "analysisInput": {
                "conversationItem": {
                    "participantId": "1",  # Identifier for the participant
                    "id": "1",  # Identifier for the conversation
                    "modality": "text",  # Specify the modality as text
                    "language": "en",  # Language of the utterance
                    "text": utterance  # The utterance to analyze
                },
                "isLoggingEnabled": False  # Disable logging for privacy
            },
            "parameters": {
                "projectName": project_name,  # Name of the deployed project
                "deploymentName": modelLabel,  # Name of the deployed model
                "verbose": True  # Enable verbose output for detailed results
            }
        }
    )

    # Print the result of the analysis
    display_result(result)

In [21]:
# Sample utterance to analyze
utterance = "I'd like to book 2 flight tickets to London and 2 rooms for the Strand Palace Hotel on Oct 15."
analyze_utterance(utterance)

Query: I'd like to book 2 flight tickets to London and 2 rooms for the Strand Palace Hotel on Oct 15.
Top Intent: Intent_CLU
Project Kind: Orchestration

Intent_CLU Details:
  Confidence Score: 0.88208646
  Sub-Intent: Book
  Sub-Intent Confidence Scores:
    - Book: 0.90206134
    - Cancel: 0.79849505
    - None: 0.7231303

Entities:
  - NumOfTickets: 2 flight tickets
    Confidence Score: 1
  - Location: London
    Confidence Score: 1
    Extra Information: [{'extraInformationKind': 'EntitySubtype', 'value': 'geography.location'}]
  - NumOfRooms: 2 rooms
    Confidence Score: 1
  - Hotel: the Strand Palace Hotel
    Confidence Score: 1
  - Location: Strand Palace Hotel
    Confidence Score: 1
    Extra Information: [{'extraInformationKind': 'EntitySubtype', 'value': 'geography.location'}]
  - Date: Oct 15
    Confidence Score: 1
    Resolutions: [{'resolutionKind': 'DateTimeResolution', 'dateTimeSubKind': 'Date', 'timex': 'XXXX-10-15', 'value': '2024-10-15'}, {'resolutionKind': 'Date

In [22]:
# Sample utterance to analyze
utterance = "What is Contoso Travel?"
analyze_utterance(utterance)

Query: What is Contoso Travel?
Top Intent: Intent_QA
Project Kind: Orchestration

Intent_QA Details:
  Confidence Score: 0.9148873

Answers:
  - Answer: Contoso Travel is a company that has been meeting the needs of business travelers since 2001. They aim to make travel as easy as possible for their clients.
    Confidence Score: 1.0
    Source: TravelFAQ.docx
    Questions: ['What is Contoso Travel?']
    Metadata: {'system_metadata_qna_edited_manually': 'true'}
    Dialog Prompts:
      - Support (ID: 68)
      - Contact (ID: 52)


In [23]:
# Sample utterance to analyze
utterance = "Reserve me 2 rooms at the Hilton in Seattle for 2 nights this June 20."
analyze_utterance(utterance)

Query: Reserve me 2 rooms at the Hilton in Seattle for 2 nights this June 20.
Top Intent: Intent_CLU
Project Kind: Orchestration

Intent_CLU Details:
  Confidence Score: 0.8362664
  Sub-Intent: Book
  Sub-Intent Confidence Scores:
    - Book: 0.87660927
    - Cancel: 0.7009587
    - None: 0.54180986

Entities:
  - NumOfRooms: 2 rooms
    Confidence Score: 1
  - Hotel: the Hilton
    Confidence Score: 1
  - Location: Seattle
    Confidence Score: 1
    Extra Information: [{'extraInformationKind': 'EntitySubtype', 'value': 'geography.location'}]
  - Date: 2 nights
    Confidence Score: 1
    Resolutions: [{'resolutionKind': 'TemporalSpanResolution', 'timex': 'P2D', 'duration': 'P2D'}]
    Extra Information: [{'extraInformationKind': 'EntitySubtype', 'value': 'datetime.duration'}]
  - Date: this June 20
    Confidence Score: 1
    Resolutions: [{'resolutionKind': 'DateTimeResolution', 'dateTimeSubKind': 'Date', 'timex': 'XXXX-06-20', 'value': '2025-06-20'}]
    Extra Information: [{'extra

In [25]:
# Sample utterance to analyze
utterance = "Cancel my reservation for 2 rooms at the Hilton in Seattle for 2 nights this June 20."
analyze_utterance(utterance)

Query: Cancel my reservation for 2 rooms at the Hilton in Seattle for 2 nights this June 20.
Top Intent: Intent_CLU
Project Kind: Orchestration

Intent_CLU Details:
  Confidence Score: 0.9170737
  Sub-Intent: Cancel
  Sub-Intent Confidence Scores:
    - Cancel: 0.9085631
    - Book: 0.69098586
    - None: 0.53636175

Entities:
  - NumOfRooms: 2 rooms
    Confidence Score: 1
  - Hotel: the Hilton
    Confidence Score: 1
  - Location: Seattle
    Confidence Score: 1
    Extra Information: [{'extraInformationKind': 'EntitySubtype', 'value': 'geography.location'}]
  - Date: 2 nights
    Confidence Score: 1
    Resolutions: [{'resolutionKind': 'TemporalSpanResolution', 'timex': 'P2D', 'duration': 'P2D'}]
    Extra Information: [{'extraInformationKind': 'EntitySubtype', 'value': 'datetime.duration'}]
  - Date: this June 20
    Confidence Score: 1
    Resolutions: [{'resolutionKind': 'DateTimeResolution', 'dateTimeSubKind': 'Date', 'timex': 'XXXX-06-20', 'value': '2025-06-20'}]
    Extra Info